In [ ]:
import numpy as np
import pandas as pd
import os
import time
import pprint
import pickle
from itertools import chain
from sklearn.ensemble import IsolationForest

pd.set_option('display.max_columns', 500)

### Preprocessing functions

In [ ]:
%run preprocessing_functions.py

### Config

In [ ]:
data_path = '../../Data'
training_name = 'train_requests.csv'
test_name = 'test_requests.csv'
target_name = 'granted_number_of_nights'
id_name = 'request_id' # None
date_features = ['answer_creation_date', 'group_creation_date', 'request_creation_date'] # None
detect_sample_size = 5000
create_csv_target = True

### Import and detect data

In [ ]:
train_sample = pd.read_csv(os.path.join(data_path, training_name),
                           sep=',', nrows=detect_sample_size, index_col=id_name)
train_sample.drop([target_name], inplace=True, axis=1)
test_sample = pd.read_csv(os.path.join(data_path, training_name),
                          sep=',', nrows=detect_sample_size, index_col=id_name)
train_test_sample = pd.concat((train_sample, test_sample), axis=0, sort=True)
column_dtypes = optimize_csv(train_test_sample, date_features)

print('\nDetected types : \n')
pprint.pprint(column_dtypes)
print('-'*20)

del train_sample, test_sample

t0 = time.time()
# Train dataset
# dateparser = lambda x: pd.to_datetime(x, format='%d%b%Y:%H:%M:%S')
train = pd.read_csv(os.path.join(data_path, training_name), sep=',', dtype=column_dtypes,
                    index_col=id_name, parse_dates=date_features or False,
                    infer_datetime_format=True,
                    error_bad_lines=False)
t1 = time.time()
# Test dataset
# dateparser = lambda x: pd.to_datetime(x, format='%d%b%Y:%H:%M:%S')
test = pd.read_csv(os.path.join(data_path, test_name), sep=',', dtype=column_dtypes,
                   index_col=id_name, parse_dates=date_features or False,
                   infer_datetime_format=True,
                   error_bad_lines=False)
t2 = time.time()

target = train[target_name]
if create_csv_target:
    pd.DataFrame(target).to_csv(os.path.join(data_path, "y_train.csv"), index=False)
    print('\ny_train.csv have been created with the target')
    print('-'*20)

train.drop([target_name], inplace=True, axis=1)
print(f'\n{target_name} have been dropped into the training set')
print('-'*20)
# requests.to_csv(os.path.join(data_path, "X_train.csv"), index=False)
# requests_test.to_csv(os.path.join(data_path, "X_test.csv"), index=False)

duplicated = train.duplicated().sum()
if duplicated > 0:
    print('\nNb of duplicated row : ' + str(duplicated))
    print('-'*20)
    
print(f'\nTime to import {training_name} : {np.round(t1 - t0, 2)}s')
print('-'*20)
print(f'Time to import {test_name} : {np.round(t2 - t1, 2)}s')
print('-'*20)
print(f'Shape of {training_name} : {train.shape}')
print('-'*20)
print(f'Shape of {test_name} : {test.shape}')

### Feature engineering

In [ ]:
ntrain = train.shape[0]
train_test = pd.concat((train, test), axis=0)
del train, test

In [ ]:
if date_features:
    for col in date_features:
        print('-'*20)
        print(col)
        print('-'*20)
        print(train_test[col].min())
        print(train_test[col].max())

#### Fillna

In [ ]:
fillna_value_num = -1
fillna_value_cat = "MiSsInG"
fillna_value_date = 0

cat_features = train_test.select_dtypes(include=['object', 'category']).columns.tolist()
num_features = train_test.select_dtypes(include=[np.number]).columns.tolist()
date_features = train_test.select_dtypes(include=['datetime']).columns.tolist()

fillna_dict = {}
for col in train_test.columns:
    if col in num_features:
        fillna_dict[col] = fillna_value_num
    elif col in date_features:
        fillna_dict[col] = fillna_value_date
    else:
        if train_test[col].dtype.name == 'category':
            train_test[col] = train_test[col].cat.add_categories([fillna_value_cat])
        fillna_dict[col] = fillna_value_cat

train_test['nb_nan'] = train_test.isnull().sum(axis=1)
train_test.fillna(value=fillna_dict, inplace=True)

assert train_test.isnull().sum().sum() == 0, 'There are missing values in the dataset'

### Aggregate features

In [ ]:
# Distinct count
agg_features = {'social_situation_id': 'group_id',
                'town': 'group_id',
                'victim_of_violence': 'group_id',
                'victim_of_violence_type': 'group_id',
                'requester_type': 'group_id',
                'request_backoffice_creator_id': 'group_id',
                'number_of_underage': 'group_id',
                'long_term_housing_request': 'group_id',
                'housing_situation_label': 'group_id',
                'group_type': 'group_id',
                'district': 'group_id',
                'child_to_come': 'group_id',
                'child_situation': 'group_id',
                'animal_presence': 'group_id'}

params = [(train_test[[by_field]+[field]].copy(),field, by_field) for
          field, by_field in agg_features.items()]

# Min and max key by value
agg_features2 = {'answer_creation_date': 'group_id',
                 'request_creation_date': 'group_id',
                 'number_of_underage': 'group_id'}

params2 = [(train_test[[by_field]+[field]].copy(),field, by_field) for
           field, by_field in agg_features2.items()]

# Most freq key by value
agg_features3 = {'answer_creation_date': 'group_id',
                 'request_creation_date': 'group_id',
                 'number_of_underage': 'group_id'}

params3 = [(train_test[[by_field]+[field]].copy(),field, by_field) for
           field, by_field in agg_features3.items()]

In [ ]:
%%time
new_features_distinct_count = do_parallel(get_distinct_count, params)
new_features_min = do_parallel(get_min, params2)
new_features_max = do_parallel(get_max, params2)
new_features_mode = do_parallel(get_mode, params3)

for col in chain(new_features_distinct_count,
                 new_features_min,
                 new_features_max,
                 new_features_mode):
    train_test = pd.concat([train_test, col], axis=1, sort=False)

In [ ]:
train_test.head()

### Date features

In [ ]:
hour_bool, days_bool, month_bool, year_bool = True, True, True, True
date_cols = train_test.select_dtypes(np.datetime64).columns

params_date = [(train_test[date_col].copy(),hour_bool, days_bool, month_bool, year_bool) for date_col in date_cols]

In [ ]:
%%time
new_features_date = do_parallel(get_date_features, params_date)

for col in new_features_date:
    train_test = pd.concat([train_test, col], axis=1, sort=False)

In [ ]:
#train_test.drop(date_cols, inplace=True, axis=1)

In [ ]:
train_test.head()

### Text features

In [ ]:
# key: text feature, value: specific tokeniser (separator)
text_feature = {'housing_situation_label': None, # custom_tokenizer,
                'group_composition_label': None}

language = 'english' # 'french'

# Text features
params_text = [(train_test[text_col].copy(), language) for text_col in text_feature.keys()]

# Tfidf features
max_features = 5
params_tfidf = [(train_test[text_col].copy(), tokenizer, language, max_features) for
                text_col, tokenizer in text_feature.items()]

In [ ]:
%%time
new_features_text = do_parallel(get_text_feature, params_text)
new_features_tfidf = do_parallel(get_tfidf_vectorizer, params_tfidf)

for col in chain(new_features_text, new_features_tfidf):
    train_test = pd.concat([train_test, col], axis=1, sort=False)

In [ ]:
#train_test.drop(text_feature.keys(), inplace=True, axis=1)

In [ ]:
train_test.head()

## Preprocessing

In [ ]:
col_to_drop = ['group_id', 'group_main_requester_id', 'housing_situation_id', 'group_composition_id']
col_to_drop.extend(list(date_cols))
# delete col_to_drop and date features
train_test.drop(col_to_drop, axis=1, inplace=True)

In [ ]:
train = train_test.iloc[:ntrain, :]
test = train_test.iloc[ntrain:, :]
del train_test

### Numerical

#### Standard scaler

In [ ]:
num_features = train.select_dtypes(include=[np.number]).columns

params_num_features = [(train[num_feature].values, test[num_feature].values) for num_feature in num_features]

In [ ]:
%%time
preproc_num_features = do_parallel(standard_scaler, params_num_features)

for col_name, preproc_num_feature in zip(num_features, preproc_num_features):
    train.loc[:, col_name] = preproc_num_feature[0]
    test.loc[:, col_name] = preproc_num_feature[1]

### Categorical

In [ ]:
cat_features = train.select_dtypes(include=['object', 'category']).columns

#### Modality grouper

In [ ]:
thresh_grouper = 50

params_modality_grouper = [(train[cat_feature].values,
                            test[cat_feature].values,
                            thresh_grouper) for cat_feature in cat_features]

In [ ]:
%%time
preproc_cat_features = do_parallel(modality_grouper, params_modality_grouper)

for col_name, preproc_cat_feature in zip(cat_features, preproc_cat_features):
    train.loc[:, col_name] = preproc_cat_feature[0]
    test.loc[:, col_name] = preproc_cat_feature[1]

#### Target encoder

In [ ]:
params_target_encoder = [(train[cat_feature].values,
                          test[cat_feature].values,
                          target.values) for cat_feature in cat_features]

In [ ]:
%%time
preproc_cat_features = do_parallel(target_encoder, params_target_encoder)

for col_name, preproc_cat_feature in zip(cat_features, preproc_cat_features):
    train.loc[:, col_name + '_target_enc'] = preproc_cat_feature[0]
    test.loc[:, col_name + '_target_enc'] = preproc_cat_feature[1]

#### One  hot encoder

In [ ]:
one_hot_limit = 10
one_hot_features = [col for col in cat_features if train[col].nunique() <= 10]
params_one_hot_encoder = [(train[cat_feature],
                           test[cat_feature],
                           cat_feature) for cat_feature in one_hot_features]

In [ ]:
%%time
preproc_cat_features = do_parallel(one_hot_encoder, params_one_hot_encoder)

for col in preproc_cat_features:
    train = pd.concat([train, col[0]], axis=1, sort=False)
    test = pd.concat([test, col[1]], axis=1, sort=False)

#### Ordinal encoder

In [ ]:
params_ordinal_encoder = [(train[cat_feature].values,
                           test[cat_feature].values) for cat_feature in cat_features]

In [ ]:
%%time
preproc_cat_features = do_parallel(ordinal_encoder, params_ordinal_encoder)

for col_name, preproc_cat_feature in zip(cat_features, preproc_cat_features):
    train.loc[:, col_name] = preproc_cat_feature[0]
    test.loc[:, col_name] = preproc_cat_feature[1]

### Delete constant cols

In [ ]:
col_constant = list(train.loc[:, train.nunique() == 1].columns)
print('constant columns: ')
print(col_constant)
train.drop(col_constant, axis=1, inplace=True)
test.drop(col_constant, axis=1, inplace=True)

### Outlier detection

In [ ]:
%%time
from sklearn.preprocessing import LabelEncoder
train_test = pd.concat([train, test], axis=0)
for col in train_test.columns:
    le = LabelEncoder()
    train_test[col] = le.fit_transform(train_test[col])
    
out = IsolationForest(n_estimators=300, max_samples=0.1, max_features=0.7, bootstrap=True,
                          n_jobs=-1, random_state=0, contamination='auto', behaviour='new')
train_test_outliers = out.fit_predict(train_test)
train['is_outlier'] = train_test_outliers[:ntrain]
test['is_outlier'] = train_test_outliers[ntrain:]

nb_outliers_train = train['is_outlier'].sum()
nb_outliers_test = test['is_outlier'].sum()

print('outliers training set: {} ({} %)'.format(nb_outliers_train,
                                                float(nb_outliers_train)/train.shape[0]))
print('outliers test set: {} ({} %)'.format(nb_outliers_test,
                                            float(nb_outliers_test)/test.shape[0]))
del train_test

### Checks

In [ ]:
assert(train.shape[0] == ntrain)
assert(train.shape[1] == test.shape[1])
assert((set(train.columns) - set(test.columns)) == set())

### Export

In [ ]:
train.to_csv(os.path.join(data_path, "X_train_prep.csv"), index=True)
test.to_csv(os.path.join(data_path, "X_test_prep.csv"), index=True)
pd.Series(cat_features).to_csv(os.path.join(data_path, "cat_cols.csv"), index=False)